In [2]:
import numpy as np
from qiskit_ibm_provider import IBMProvider
from itertools import product

In [10]:
# IBMProvider.save_account(token=<INSERT_IBM_QUANTUM_TOKEN>)
# provider = IBMProvider()
# job_id   = "cp4dpjambjng008jmwvg"
# result   = provider.retrieve_job(job_id)

In [5]:
backends = ["ibm_cairo", "ibmq_kolkata"]

for abbrev, backend in product(["BB-QCD", "TT-QCD"], backends):
    result = np.load(f"ibmq/{backend}_r0-p_2_n25_b300-3_1_3.npy", allow_pickle=True)
    result = result.item()[abbrev]
    print(f"{abbrev} in backend {backend}:")
    print(f"IBMQ Frobinius norm = {np.linalg.norm(result['meas_penl']-result['meas_ibmq'])}")
    print(f"QASM Frobinius norm = {np.linalg.norm(result['meas_penl']-result['meas_qasm'])}")
    print("-"*100)